<a href="https://colab.research.google.com/github/Lucas-Welter/Twitter_Opinion_Scrapper/blob/main/Twitter_Opinion_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **COLETANDO DADOS**

In [ ]:
import numpy as np

In [ ]:
# pegando tabela do drive
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('eleicoes2022').sheet1


In [ ]:
# convertendo para um dataframe
rows = worksheet.get_all_values()
import pandas as pd
df=pd.DataFrame(rows[1:],columns=rows[0])

In [ ]:
# palavras chave para o filter
sentNeu=['neu']
sentPos=['pos']
sentNeg=['neg']

In [ ]:
# Armazenando os valores da tabela antiga e filtrando .
coluna_neu = df['Texto'].loc[df['Classe'].isin(sentNeu)]
coluna_neu = list(filter(('NaN').__ne__, coluna_neu))
coluna_pos = df['Texto'].loc[df['Classe'].isin(sentPos)]
coluna_pos = list(filter(('NaN').__ne__, coluna_pos))

# OBS: Os pop's são para tirar o valor excessivo que estava desbalanceando o Data set
for i in range(5):
  coluna_pos.pop()
coluna_neg = df['Texto'].loc[df['Classe'].isin(sentNeg)]
coluna_neg = list(filter(('NaN').__ne__, coluna_neg))
for i in range(293):
  coluna_neg.pop()



In [ ]:
# Criando um novo DataFrame para não ter que lidar com os índices residuais do anterior
df2=pd.DataFrame()

In [ ]:
# Inserindo as colunas no novo DataFrame
df2.insert(loc=0,column='NEU',value= coluna_neu)
df2.insert(loc=0,column='POS',value= coluna_pos)
df2.insert(loc=0,column='NEG',value= coluna_neg)

In [ ]:
df2

In [ ]:
# Função para retirar os caracteres especiais.
for column in df2.columns:
  df2[column] = df2[column].str.replace(r'\W'," ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Estabelecendo conjunto de treino global
intencoes = ['NEU', 'POS', 'NEG']
xtrain_global = []
ytrain_global = []
for intencao in intencoes:
    lintencao = df2[intencao].values.tolist()
    xtrain_global += lintencao
    ytrain_global += [intencao]*len(lintencao)


In [ ]:
xtrain_global,ytrain_global

## SLPIT ENTRE TREINO, TESTE E VALIDAÇÃO

In [ ]:
import sklearn.model_selection as model_selection

In [ ]:
# Dividi o treino em treino(80%) e teste(20%), após isso dividi o teste em teste(10%) e validação(10%)
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(xtrain_global, ytrain_global, test_size=0.20, random_state=42,shuffle=True)
xtestS, xval, ytesteS, yval = model_selection.train_test_split(xtest, ytest, test_size=0.50, random_state=42,shuffle=True)

In [ ]:
# Verificando se foi dividio corretamente
len(xtrain_global)


22302

In [ ]:
len(xtestS)

2230

In [ ]:
len(xval)

2231

### **PRÉ-PROCESSAMENTO**

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
converter = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [ ]:
xtrain_emb = converter.encode(xtrain)
xval_emb = converter.encode(xval)
xtestS_emb = converter.encode(xtestS)

### **INDUÇÃO DO MODELO**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
# Escolhendo o melhor K com o sklearn

In [ ]:
training_accuracy=[]
test_accuracy=[]
# O google me barrou antes de eu conseguir executar esse teste(range 1,30), mas meu melhor valor anteriormente foi 17
neighbours=range(1,30)

for number_of_neighbours in neighbours:
  KNN =KNeighborsClassifier(n_neighbors=number_of_neighbours)
  KNN.fit(xtrain_emb,ytrain)
  training_accuracy.append(KNN.score(xtrain_emb,ytrain))
  test_accuracy.append(KNN.score(xtestS_emb,ytesteS))

In [ ]:
# Demonstrando em gráfico o melhor k.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(neighbours,training_accuracy,label="training accuracy")
plt.plot(neighbours,test_accuracy,label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("neighbors")
plt.legend()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=17)

In [ ]:
knn.fit(xtrain_emb,ytrain)

In [ ]:
knn.predict(converter.encode(['lULA É LADRÃO','vou votar no Bolsonaro 22','é um ótimo dia para ser brasileiro']))

### **AVALIAÇÃO DO MODELO**

In [ ]:
import sklearn.metrics as metrics

In [ ]:
pred = knn.predict(xtestS_emb)

In [ ]:
print(metrics.classification_report(ytesteS,pred))

In [ ]:
print(metrics.confusion_matrix(ytesteS,pred))

### **MODELO FINAL**

In [ ]:
knn.fit(converter.encode(xtrain_global),ytrain_global)

In [ ]:
from joblib import dump, load

In [ ]:
dump(knn, 'SensorTwitter.joblib') 

In [ ]:
modelo_final = load('SensorTwitter.joblib') 

In [ ]:
modelo_final.predict([converter.encode('simone ladrão')])

### **CONSTRUÇÃO DO SENSOR**

In [ ]:
!pip install snscrape
import snscrape.modules.twitter as sns

In [ ]:
# Utilizando o query para extrair os tweets (reaproveitei da parte 1)
query_bolsonaro = "(bolsonaro) lang:pt"
tweets_bolsonaro = []
query_lula = "(lula) lang:pt"
tweets_lula = []
query_ciro = "(ciro) lang:pt"
tweets_ciro = []
query_simone = "(simone tebet) lang:pt"
tweets_simone = []
max_size = 149

In [ ]:
# Função que retorna a resposta do modelo sobre os tweets, que inclusive também
# tem os caracteres especiais removidos nessa parte
def getTweets(query, tweets):
    for tweet in sns.TwitterSearchScraper(query).get_items():
        tweets.append(tweet.content)
        if len(tweets) > max_size:
            break
    tweets.str.replace(r'\W'," ")        
    grafico=[]        
    grafico = modelo_final.predict(converter.encode(tweets))
    return grafico


In [ ]:
 ##OBS: CASO DE ERRO, TENTE MAIS ALGUMAS VEZES, PODE SER QUE O TWEET SEJA INVÁLIDO
result_bolsonaro = getTweets(query_bolsonaro, tweets_bolsonaro)
result_lula = getTweets(query_lula, tweets_lula)
result_ciro = getTweets(query_ciro, tweets_ciro)
result_simone = getTweets(query_simone, tweets_simone)

In [ ]:
# Classes para o gráfico
classes=['pos','neu','neg']

In [ ]:
# Função que conta os sentimentos positivo, negativo e neutro e devolve o gráfico plotado para aquele candidato
def geradorDeGrafico(resultado):
  cont=0
  resultado
  for elemento in resultado:
    if elemento == 'POS':
      cont+=1
  tweetPositivo = cont
  cont=0
  for elemento in resultado:
    if elemento == 'NEU':
      cont+=1
  tweetNegativo = cont
  cont=0
  for elemento in resultado:
    if elemento == 'NEG':
      cont+=1
  tweetNeutro = cont
  cont=0
  todos_tweets = [tweetPositivo,tweetNeutro,tweetNegativo]
  plt.bar(classes,todos_tweets)
  

In [ ]:
!pip install schedule
import schedule
import time


In [ ]:
print("GRÁFICO DO BOLSONARO")
geradorDeGrafico(result_bolsonaro)

In [ ]:
print("GRÁFICO DO LULA")
geradorDeGrafico(result_lula)

In [ ]:
print("GRÁFICO DO CIRO")
geradorDeGrafico(result_ciro)

In [ ]:
print("GRÁFICO DA SIMONE")
geradorDeGrafico(result_simone)